# 용의자 대화 녹음기 (메모리)

- 사용자가 입력한 정보는 LLM에 전달하여 답변을 받아 오고 UI 채팅창을 통해 출력된다.
- 사용자가 **"기록 시작" , "녹음 시작"** 지시를 하면 이후의 대화 내용은 기억하고, 그 이전의 내용은 기억하지 않는다.
- 사용자가 **"기록 중지" , "녹음 중지"** 지시를 하면 그 이후의 내용은 기억하지 않는다.
- 사용자가 **"기록 삭제" , "녹음 삭제"** 지시를 하면 모든 내용은 삭제한다.
- Gradio UI를 통해 녹음기의 상태를 표시한다.
- memory는 현재 세션에서만 캐시 형태로 사용될 수 있도록 관리한다.

**※ 대화 기록이 있는 경우, 기록이 없는 경우에 대화 요약을 요청하여 결과를 비교한다.**

## 환경 설정 및 OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv

from typing import List
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage, AIMessage
from langchain.tools import tool

import gradio as gr

# .env 파일 로드
load_dotenv()

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## 상태 관리 (메모리)

`BaseModel`과 `TypedDict` 사용한 상태 클래스 정의 방식 차이 비교

| 항목               | `BaseModel` (Pydantic 기반)                 | `TypedDict` (표준 타입 지정)       |
| ---------------- | ----------------------------------------- | ---------------------------- |
| **용도**           | 유효성 검사 및 메서드 포함 모델 정의                     | 단순 타입 명세 (데이터 구조 정의용)        |
| **런타임 검증**       | ✅ 지원 (필드 누락, 타입 오류 등 검사 가능)               | ❌ 없음 (정적 타입 확인용)             |
| **기본값 지정**       | ✅ `Field(...)`, `default_factory` 등 사용 가능 | ❌ 직접 지정 불가 (초기화 시 수동 지정 필요)  |
| **메서드 정의**       | ✅ 클래스 내부 메서드 사용 가능 (`get_status()` 등)     | ❌ 불가                         |
| **LangGraph 호환** | ✅ 가능하지만 `add_messages` 사용 제한              | ✅ 권장됨 (LangGraph에서 공식 지원 방식) |
| **가독성/기능성**      | 좋음 (유형 체크 + 메서드 포함 가능)                    | 가볍고 명확한 구조화에 적합              |
| **예시 사용 사례**     | 복잡한 상태 관리, LLM 메시지 추적                     | LangGraph 상태 딕셔너리 구조         |

`class State(BaseModel)`:
- 메서드 정의 가능 (`get_status`)
- 필드에 대한 설명과 기본값(`default_factory=list` 및 `bool = False`) 포함
- LangGraph에서 사용할 때는 `Annotated[..., add_messages]`를 직접 적용할 수 없음

In [ ]:
class State(BaseModel):
    """대화 목록과 기록 상태를 저장하는 모델"""
    is_recording: bool = False
    memory: List[HumanMessage | AIMessage] = Field(description="사용자와 AI의 대화 목록", default_factory=list)

    def get_status(self) -> str:
        """현재 기록 상태를 문자열로 반환"""
        return "🔴 기록 중" if self.is_recording else "⚫️ 기록 대기"

## 메모리 관리 도구 (tool)

`BaseModel`과 `TypedDict`으로 정의한 상태 클래스 속성 참조 방식 비교
| 항목        | `BaseModel` (`state.is_recording`) | `TypedDict` (`state["is_recording"]`) |
| --------- | ---------------------------------- | ------------------------------------- |
| 속성 참조     | `.` 점 표기법 사용                       | `[]` 키 딕셔너리 방식 사용                     |
| 예외 처리     | 존재하지 않는 속성 → `AttributeError`      | 존재하지 않는 키 → `KeyError`                |
| IDE 자동완성  | ✅ 지원됨                              | ❌ 대부분 지원 안 됨                          |
| 타입 안정성    | ✅ 런타임 유효성 검사 포함                    | ❌ 없음 (정적 타입 검사만)                      |
| 가독성       | 높음 (`state.is_recording`)          | 다소 낮음 (`state["is_recording"]`)       |
| 메서드 포함 여부 | ✅ 가능 (`state.get_status()`)        | ❌ 메서드 정의 불가                           |
- BaseModel에서도 `["key"]` 방식도 지원되지만, `.` 점 표기법이 더 직관적이고 IDE 자동완성 및 정적 분석에서 유리
- 동적 속성 접근이 필요하지 않다면 가독성과 안전성 측면에서 `state["is_recording"]` 보다는 `state.is_recording` 방식을 사용하는 것을 권고

In [ ]:
@tool
def start_memory_recording(state: State) -> str:
    """사용자가 대화 기록 시작을 요청할 때 이 도구를 호출합니다. '기록 시작', '녹음 시작' 등의 명령어에 해당합니다."""
    state.is_recording = True
    return "지금부터 대화를 기록합니다."

@tool
def stop_memory_recording(state: State) -> str:
    """사용자가 대화 기록 중지를 요청할 때 이 도구를 호출합니다. '기록 중지', '녹음 중지' 등의 명령어에 해당합니다."""
    state.is_recording = False
    return "지금부터 대화를 기록하지 않습니다."

@tool
def clear_all_memory(state: State) -> str:
    """사용자가 모든 대화 기록 삭제를 요청할 때 이 도구를 호출합니다. '기록 삭제', '녹음 삭제' 등의 명령어에 해당합니다."""
    state.memory.clear()
    return "모든 대화 기록을 삭제했습니다."

## 프롬프트 구성
1. `system_prompt`: 프롬프트 템플릿(`ChatPromptTemplate`)에 고정적으로 포함된 부분
    - AI의 역할이나 행동 지침을 정의하는 정적인 텍스트로, 대화가 진행되어도 변하지 않음.

2. `chat_history (state.memory)`: `MessagesPlaceholder를` 통해 동적으로 채워지는 대화 기록
    - `clear_all_memory` 함수는 바로 이 `state.memory` 리스트의 내용만 삭제.

3. 따라서 '기록 삭제' 명령을 실행하면 `state.memory` 리스트는 비워지지만, `system_prompt`는 프롬프트 템플릿 구조의 일부로서 그대로 유지
    - 이후 새로운 대화를 시작할 때도 시스템 프롬프트는 항상 LLM에 전달됨.

`MessagesPlaceholder`의 위치에 따른 LLM 응답 방식의 영향

| 구조       | 순서                       | 효과                                 |
| -------- | ------------------------ | ---------------------------------- |
| ✅ 중간(권장) 구조  | `system → chat_history  → human` | 시스템 지침이 일관성 있게 적용됨                 |
| ⚠️ 맨 앞 구조 | `chat_history → system → human` | 시스템 지침이 늦게 적용되어 무시되거나 영향을 덜 줄 수 있음 |
| ⚠️ 맨 뒤 구조 | `system → human → chat_history` | 과거 대화를 나중에 참조 → 잘못된 흐름 유도 |

In [ ]:
system_prompt = """
당신은 명탐정 코난이 수사하는 사건의 유력한 용의자 입니다.
탐정의 질문에 적절하게 답변을 해야 합니다.
탐정은 대화 기록을 관리하는 녹음기를 사용할 수 있습니다.: 'start_memory_recording', 'stop_memory_recording', 'clear_all_memory'.
사용자의 요청을 분석해서 적절한 도구를 사용할 수 있도록 허용해 주어야 합니다.
예를 들어, 사용자가 '녹음을 시작합니다.' 라고 말하면 'start_memory_recording' 도구를 호출해야 합니다.
사용자가 대화 요약을 요청하면 대화 이력을 이용해서 요약을 제공합니다. 
그 외 모든 일반적인 대화에는 도구를 사용하지 말고 직접 답변해야 합니다.
답변을 한 후에 절대로 추가적인 질문을 해서는 안됩니다.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

## LCEL 파이프라인 설정

In [ ]:
# 도구 목록 설정
tools = [start_memory_recording, stop_memory_recording, clear_all_memory]

# 도구 이름을 키로, 함수를 값으로 하는 딕셔너리를 생성
tool_map = {tool.name: tool for tool in tools}

# llm에 도구들을 바인딩
llm_with_tools = llm.bind_tools(tools)

# LCEL 체인을 구성
chain = prompt | llm_with_tools

## UI와 LLM 인터페이스 함수

사용자의 입력을 받아 AI 응답을 처리하고, UI와 상태를 업데이트
1. LLM 응답이 도구 호출인 경우 도구가 사용 현황을 UI 출력으로 표시하고 메모리에는 기록하지 않음
2. 메모리가 기록 상태일때만 `chat_history` 공간인 `state.memory` 에 대화 내용을 기록
- `state.memory` 배열 함수 `append()`와 `extend()`의 구분

    | 항목    | `append(...)`                            | `extend([...])`                        |
    | ----- | ---------------------------------------- | -------------------------------------- |
    | 동작    | 리스트에 **하나의 요소**를 추가                      | 리스트에 **여러 요소**를 한꺼번에 추가                |
    | 결과    | 중첩 리스트 가능성 있음                            | 낱개의 요소가 나란히 추가됨                        |
    | 예시    | `[A, B].append([C, D]) → [A, B, [C, D]]` | `[A, B].extend([C, D]) → [A, B, C, D]` |
    | 사용 상황 | 한 번에 메시지 1개만 추가할 때                       | 여러 메시지를 순차적으로 추가할 때                    |


* 리턴값의 `state.get_status()`는 UI 레이아웃의 `status_indicator`의 입력값으로 전달됨

In [ ]:
def chat_fn(user_input: str, history: list, state: State):
    response = chain.invoke({"input": user_input, "chat_history": state.memory})

    print(response)
    
    # UI 챗봇 창에 출력할 LLM 메시지
    ai_output = ""

    # LLM 응답이 도구 호출일 경우
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_to_run = tool_map.get(tool_call["name"])
            if tool_to_run:
                ai_output = tool_to_run.invoke({"state": state})
            else:
                ai_output = f"오류: '{tool_call['name']}' 도구를 찾을 수 없습니다."
    # 일반 메시지일 경우
    else:
        ai_output = response.content

    # history는 UI 표시용, state.memory는 LLM 전달용
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": ai_output})

    # 기록 상태일 때만 실제 메모리에 대화 내용 추가
    if state.is_recording:
        # 도구 호출이 아닌 일반 대화만 메모리에 저장
        if not response.tool_calls:
            state.memory.extend(
                [HumanMessage(content=user_input), AIMessage(content=ai_output)]
            )

    return history, state, state.get_status()

### Gradio UI

※ 입력 처리 상세 설명

1. `user_input.submit(chat_fn, inputs=[user_input, state], outputs=[chatbot, state])`
    - `user_input.submit(...)` : 사용자가 텍스트박스에 입력하고 Enter를 누르면 실행
    - `chat_fn` : 메인 처리 함수 - 사용자의 입력을 받아 챗봇 응답과 상태를 계산 또는 
    - `inputs=[user_input, state]` : 텍스트 입력값과 이전 상태(state)를 chat_fn에 전달
    - `outputs=[chatbot, state]` : 함수 결과로 나온 대화 내용과 상태를 챗봇 창과 내부 상태에 각각 반영

2. `user_input.submit(lambda: "", None, user_input)`
    - `lambda: ""` : 아무 동작 없이 빈 문자열을 반환하는 함수
    - `None` : 입력값은 없음
    - `user_input` : 출력 대상은 텍스트박스 자신 - 입력 후 자동으로 입력창을 초기화

※ 람다(`lambda`) 함수는 이름 없이 간단하게 사용하는 **한 줄짜리 익명 함수(anonymous function)** - 임시로 사용할 간단한 기능이 필요할 때 구현

```python
    lambda 매개변수1, 매개변수2, ... : 표현식
```

In [ ]:
# Gradio UI 레이아웃 설계
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    initial_state = State()
    state = gr.State(value=initial_state)

    gr.Markdown("### 🕵️‍♂️ 탐정의 녹음기!")
    gr.Markdown("`기록 시작`, `기록 중지`, `기록 삭제` 명령어를 사용해 보세요.")

    with gr.Row():
        status_indicator = gr.Textbox(
            value=initial_state.get_status(),
            label="메모리 상태",
            interactive=False,
        )

    chatbot = gr.Chatbot(label="대화창", height=300, type="messages")

    user_input = gr.Textbox(placeholder="어제 저녁 9시에 어디에 계셨죠?", show_label=False)
    user_input.submit(
        chat_fn,
        inputs=[user_input, chatbot, state],
        outputs=[chatbot, state, status_indicator]
    )

    # 입력창 초기화
    user_input.submit(lambda: "", None, user_input)

# 웹 서버 실행
demo.launch()

-----
** End of Documents **